In [21]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('../carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


import carla

import time
import math
import hydra
import logging
import argparse


from numpy import random
from omegaconf import DictConfig, OmegaConf, ValidationError

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.pgm import PGM
from utils.weather import Weather
from core.vehicle_manager import VehicleManager
from core.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints, hour_to_altitude

from hydra.core.config_store import ConfigStore
from schemas.pgm_schema import PGMModel
from schemas.weather_schema import WeatherSchema, SunSchema
from schemas.config_schema import ConfigSchema

In [17]:
from pgmpy.readwrite import XMLBIFReader


file_path = "eCARLA_PGM.xml"
reader = XMLBIFReader(file_path)
model = reader.get_model()

In [19]:
reader.get_variables()

['clouds', 'fog', 'pedestrians', 'rain', 'speed', 'time', 'traffic']

In [20]:
reader.get_states()

{'clouds': ['heavy', 'light', 'moderate', 'no'],
 'fog': ['heavy', 'light', 'moderate', 'no'],
 'pedestrians': ['high', 'low', 'medium'],
 'rain': ['heavy', 'light', 'moderate', 'no'],
 'speed': ['high', 'low', 'medium'],
 'time': ['afternoon',
  'early_evening',
  'early_morning',
  'evening',
  'late_evening',
  'late_morning',
  'morning',
  'night'],
 'traffic': ['heavy', 'low', 'medium']}

In [12]:
def time_class(row):
    time = row['hour']
    if time >= 2 and time <= 6:
        return 'early morning'
    elif time > 6 and time <= 9:
        return 'morning'
    elif time > 9 and time <= 12:
        return 'late morning'
    elif time > 12 and time <= 17:
        return 'afternoon'
    elif time > 17 and time <= 19:
        return 'early evening'
    elif time > 19 and time <= 21:
        return 'evening'
    elif time > 21 and time <= 23:
        return 'late evening'
    else:
        return 'night'

In [13]:
def altitude_to_time(angle):
    if angle >= -60 and angle <= 0:
        return 'early morning'
    elif angle > 0 and angle <= 45:
        return 'morning'
    elif angle > 45 and angle <= 90:
        return 'late morning'
    elif angle < 90 and angle >= 15:
        return 'afternoon'
    elif angle < 15 and angle >= -15:
        return 'early evening'
    elif angle < -15 and angle >= -45:
        return 'evening'
    elif angle < -45 and angle >= -75:
        return 'late evening'
    else:
        return 'night'

In [14]:
for i in range(24):
    print(f"Hour {i} : {hour_to_altitude(i)}: {altitude_to_time(hour_to_altitude(i))}")

Hour 0 : -90: night
Hour 1 : -75: late evening
Hour 2 : -60: early morning
Hour 3 : -45: early morning
Hour 4 : -30: early morning
Hour 5 : -15: early morning
Hour 6 : 0: early morning
Hour 7 : 15: morning
Hour 8 : 30: morning
Hour 9 : 45: morning
Hour 10 : 60: late morning
Hour 11 : 75: late morning
Hour 12 : 90: late morning
Hour 13 : 75: late morning
Hour 14 : 60: late morning
Hour 15 : 45: morning
Hour 16 : 30: morning
Hour 17 : 15: morning
Hour 18 : 0: early morning
Hour 19 : -15: early morning
Hour 20 : -30: early morning
Hour 21 : -45: early morning
Hour 22 : -60: early morning
Hour 23 : -75: late evening


In [ ]:
from pgmpy.readwrite import XMLBIFReader

reader = XMLBIFReader("asia.xml")
read_model = reader.get_model()
read_model.get_cpds()

In [15]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    pgm = PGM(cfg)
    pgm_variables = pgm.get_variables()
    pgm_states = pgm.get_states()

    # get rain state based on weather
    rain_value = cfg.weather.rain
    rain_state = None
    for s in pgm_states.Rain:
        if (rain_value <= s.value):
            rain_state = s.name
            break

    print(rain_state)

    traffic_state = pgm.predict_state(
        [pgm_variables.Traffic, pgm_variables.Speed, pgm_variables.Pedestrians], 
        evidence={
            pgm_variables.Rain: rain_state
        }
    )

    rng = random.default_rng()
    num_of_vehicles = pgm_states.Traffic[traffic_state[pgm_variables.Traffic]].value
    num_of_vehicles = rng.poisson(num_of_vehicles)
    print(f"Spawning {num_of_vehicles} vehicles.")

    speed = pgm_states.Speed[traffic_state[pgm_variables.Speed]].value
    print(f"Maximum speed is {speed}% of the speed limit.")

    num_of_pedestrians = pgm_states.Pedestrians[traffic_state[pgm_variables.Pedestrians]].value
    num_of_pedestrians = rng.poisson(num_of_pedestrians)
    print(f"Spawning {num_of_pedestrians} pedestrians.")

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    # print(OmegaConf.to_yaml(vcfg, resolve=True))
    tm = VehicleManager(client=client, cfg=vcfg, asynch=cfg.asynch)

    # v = world.player.get_velocity()
    # 'Speed:   % 15.0f km/h' % (3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)),
    speed_history = list()

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    # print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg, asynch=cfg.asynch)
    try:
        world = client.get_world()

        all_vehicles = tm.spawn_vehicles(number_of_vehicles=num_of_vehicles)
        pm.spawn_walkers(number_of_walkers=num_of_pedestrians, percentagePedestriansRunning=pcfg.running_percentage, percentagePedestriansCrossing=pcfg.crossing_percentage)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())
        weather.setWeather(cfg.weather)
        world.set_weather(weather.weather)

        elapsed_time = 0.0

        tm.update_global_speed(speed)

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
            v = world.get_actor(all_vehicles[0]).get_velocity()
            current_speed = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
            speed_history.append(current_speed)
            # print('Speed:   % 15.0f km/h' % (current_speed))
    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)
        return speed_history

In [ ]:
try:
    cs = ConfigStore.instance()
    cs.store(group="weather", name="base_weather_model", node=WeatherSchema)
    cs.store(group="weather/sun", name="base_sun_model", node=SunSchema)
    cs.store(group="model", name="base_pgm_model", node=PGMModel)
    cs.store(name="base_config_model", node=ConfigSchema)
    with hydra.initialize(version_base=None, config_path="../conf"):
        cfg = hydra.compose(
            config_name="config", 
            overrides=[
                "weather=clear",
                "weather/sun=night"
                
                ])
        print(cfg.weather)
        clear_speeds = main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')